In [1]:
# @title Preparation
#Drive Link materiale: https://drive.google.com/drive/folders/10T3u5qfomOPUeXfHPEzeA3GRiT-ZqgBk?usp=sharing
!pip install tensorflow-gpu==1.15
!pip install scikit-learn==0.20.1

!pip install -q keras-bert keras-rectified-adam
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

     |████████████████████████████████| 411.5MB 34kB/s 
     |████████████████████████████████| 512kB 42.2MB/s 
     |████████████████████████████████| 3.8MB 42.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4492f7ac3b1da6268b67430cbed583d751aeb23a8a12de2280cc87a0c451c2d0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc4 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.

In [2]:
import tensorflow as tf
import keras
from keras_radam import RAdam
from keras_bert import get_custom_objects
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer
import pandas as pd
import tensorflow.keras.backend as K
import sys
from sklearn.metrics import classification_report
from google.colab import drive

Using TensorFlow backend.


In [0]:
# @title Constants
np.random.seed(42)
SEQ_LEN = 128
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-5

In [0]:
# @title Environment
import os
pretrained_path = '/content/drive/My Drive/codiesp/'
config_path = os.path.join(pretrained_path, 'config.json')
checkpoint_path = os.path.join(pretrained_path, 'model.ckpt-2000000')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [5]:
# @title Load Basic Model
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from run_classifier import *
import modeling
import optimization
import tokenization

import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 9.10 MiB/s, done.
Resolving deltas: 100% (185/185), done.



In [6]:
# @title Load Data
!pip install category_encoders==1.3.0
import joblib
import pandas as pd
from keras import Sequential
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import random
from keras.layers import Input
import keras
from keras.layers import Conv1D , Embedding
from keras.layers import Dropout
from keras.layers import MaxPool1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import GlobalMaxPool1D
from keras.layers import Bidirectional
import category_encoders as ce

from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

def remove_symbol(s):
    s = s.replace(",", "")
    s = s.replace(".", "")
    s = s.replace(";", "")
    s = s.replace(":", "")
    s = s.replace("_", "")
    s = s.replace("+", "")
    s = s.replace("ª", "")
    s = s.replace("-", "")
    s = s.replace("<", "")
    s = s.replace(">", "")
    s = s.replace("!", "")
    s = s.replace("?", "")
    s = s.replace("(", "")
    s = s.replace(")", "")
    s = s.replace("[", "")
    s = s.replace("]", "")
    s = s.replace("'", "")
    s = s.replace("0", "")
    s = s.replace("1", "")
    s = s.replace("2", "")
    s = s.replace("3", "")
    s = s.replace("4", "")
    s = s.replace("5", "")
    s = s.replace("6", "")
    s = s.replace("7", "")
    s = s.replace("8", "")
    s = s.replace("9", "")
    s = s.replace("%", "")
    s = s.strip()
    s = s.lower()
    return s

# Rimozione delle stopword
def clar_text(text):
    t = remove_symbol(str(text).strip().lower())
    tokens = list(str(text).lower().split(" "))
    for z in range(0, len(stop_word)):
        if stop_word[z] in tokens:
            while stop_word[z] in tokens:
                tokens.remove(str(stop_word[z]))

    tt = ""
    for it in tokens:
      tt = tt +" "+it
    return tt

def _pad(input_ids, max_seq_len):
    x = []
    input_ids = input_ids[:min(len(input_ids), max_seq_len - 2)]
    input_ids = input_ids + [0] * (max_seq_len - len(input_ids))
    return np.array(input_ids)

#LOADING DATASET
df = pd.read_csv('/content/drive/My Drive/codiesp/TestSet.csv')
df = df[['id', 'Desc']]
#df = df.sample(frac=1).reset_index(drop=True)
# df = df[pd.notnull(df['desc'])]
#print(df.head(10))
print(df.shape)

df.index = range(df.shape[0])
print("Parole: " + str(df['Desc'].apply(lambda x: len(x.split(' '))).sum()))  # ci sono circa 211456 parole

# rimozione SOLO dei simboli (nessuno stemming e nessuna rimozione delle stopword)
df['Desc'] = df['Desc'].apply(remove_symbol)
#print(df.head(10))

# Acquisizione delle stop word
file_stopw = open("/content/drive/My Drive/codiesp/stop_word.pck", "rb")
stop_word = pickle.load(file_stopw)
df['Desc'] = df['Desc'].apply(clar_text)

#suddivisione train_test
test = df
# train_test_split(df, test_size=0.3, random_state=42)

#prepare class encoder
#le = ce.OneHotEncoder(return_df=False, impute_missing=False, handle_unknown="ignore")
#labels = le.fit(list(df['Code']))
mapa = [0,1]

labels_map = [0,1]
#i = 0
#for a in mapa:
#    labels_map.append(a)
#print(labels_map)

#Tokenization
#Inizialize the tokenizer
tokenizer = tokenization.FullTokenizer(vocab_path, do_lower_case=True)
#indices_train = []
indices_test = []

#for text in train['Desc']:
#  tk = tokenizer.tokenize(text)
#  tokens = ["[CLS]"] + tk + ["[SEP]"]
#  token_ids = tokenizer.convert_tokens_to_ids(tokens)
#  token_ids = _pad(token_ids,SEQ_LEN)
#  indices_train.append(token_ids)

for text in test['Desc']:
  tk = tokenizer.tokenize(text)
  tokens = ["[CLS]"] + tk + ["[SEP]"]
  token_ids = tokenizer.convert_tokens_to_ids(tokens)
  token_ids = _pad(token_ids,SEQ_LEN)
  indices_test.append(token_ids)

#indices_train = [indices_train, np.zeros_like(indices_train)]
indices_test= [indices_test, np.zeros_like(indices_test)]

ids = test['id']
print(ids)
#train_labels = train['Code']
#train_labes_indexes = []
#for label in train_labels:
#  if(label =='emp'):
#    train_labes_indexes.append(0)
#  else:
#    train_labes_indexes.append(1)


     |████████████████████████████████| 61kB 2.7MB/s 
(53220, 2)
Parole: 1131000

0        S1137-66272016000300011-1
1        S1137-66272016000300011-1
2        S1137-66272016000300011-1
3        S1137-66272016000300011-1
4        S1137-66272016000300011-1
                   ...            
53215    S1130-01082017000800012-1
53216    S1130-01082017000800012-1
53217    S1130-01082017000800012-1
53218    S1130-01082017000800012-1
53219    S1130-01082017000800012-1
Name: id, Length: 53220, dtype: object


In [7]:
print(len(test))

53220


In [8]:
!pip install joblib
import joblib

labels_map = joblib.load('/content/drive/My Drive/codiesp/labels_map_bert_only_01.joblib')

In [9]:
bert = load_trained_model_from_checkpoint(
    config_file=config_path,
    checkpoint_file=checkpoint_path,
    training=True,
    trainable=True,
    seq_len=128
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# @title Build Custom Model

inputs = bert.inputs[:2]
dense = bert.get_layer('NSP-Dense').output
dense1 = keras.layers.Dense(units=1000, activation='tanh') (dense)
outputs = keras.layers.Dense(units=2, activation='softmax')(dense1)
modelk = keras.models.Model(inputs, outputs)
modelk.compile(
    RAdam(lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [11]:
# @title Initialize Variables
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# @title Fit

#filepath="/content/drive/My Drive/codiesp/bert_only_01.{epoch:05d}-{val_loss:.5f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

#    checkpoint
#callbacks_list = [
#]

#modelk.fit(
#    indices_train,
#    train_labes_indexes,
#    epochs=50,
#    batch_size=32,
#    validation_split = 0.10,
#    callbacks=callbacks_list
#)

In [0]:
modelk.load_weights('/content/drive/My Drive/codiesp/bert_only_01.00003-0.14749.hdf5')

In [14]:
# @title Predict
predicts = modelk.predict(indices_test, verbose=True)
print(predicts[0])

53220/53220 [==============================] - 248s 5ms/step
[0.16253346 0.8374665 ]


In [15]:
res_encoded = []
for a in predicts:
    val = a.argmax()
    res_encoded.append(labels_map[val])

print(res_encoded)
#res_labels = le.inverse_transform(DataFrame.res_encoded)

#test_labels = test['Code']
#test_labes_indexes = []
#for label in test_labels:
#  if label == 'emp':
#    test_labes_indexes.append(0)
#  else:
#    test_labes_indexes.append(1)



#print('Testing accuracy %s' % accuracy_score(test_labes_indexes, res_encoded))
#print('Testing F1 score: {}'.format(f1_score(test_labes_indexes, res_encoded, average='weighted')))


[1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 

In [0]:
#!pip install scikit-learn
#from sklearn.metrics import classification_report

In [0]:
#print(classification_report(test_labes_indexes, res_encoded,digits=5))

#CLASSIFICATION pathologies with their numbers


In [18]:
#Estraggo elementi di cui si crede ci sia la patologia
left_to_classify = []
#left_to_classify_labels = []
left_to_classify_ids = []
res_encoded_original = res_encoded.copy()

print(test.head(10))
test_desc = test['Desc']
test_bal = test['id']

print(test_bal[0])
i = 0
for item in res_encoded:
  if item == 1:
    left_to_classify.append(test_desc[i])
    left_to_classify_ids.append(test_bal[i])
  i = i+1

print(len(left_to_classify))
print(len(left_to_classify_ids))

joblib.dump(left_to_classify,'/content/drive/My Drive/codiesp/left_to_classify_Task1.vec')
joblib.dump(left_to_classify_ids,'/content/drive/My Drive/codiesp/left_to_classify_ids_Task1.vec')

                          id                                               Desc
0  S1137-66272016000300011-1   varón  años consultó dolor tumefacción borde ...
1  S1137-66272016000300011-1                     refería antecedente traumático
2  S1137-66272016000300011-1        dolor carácter espontáneo inicio progresivo
3  S1137-66272016000300011-1   dicho dolor inicialmente cedía aine increment...
4  S1137-66272016000300011-1   exploración física apreció tumefacción mal de...
5  S1137-66272016000300011-1                 enrojecimiento aumento calor local
6  S1137-66272016000300011-1                  movilidad muñeca dedos conservada
7  S1137-66272016000300011-1   radiografía simple mostraba lesión ósea radio...
8  S1137-66272016000300011-1   completó estudio gammagrafía ósea tac resonan...
9  S1137-66272016000300011-1   gammagrafía ósea tres fases realizada tecneci...
S1137-66272016000300011-1
28436
28436


['/content/drive/My Drive/codiesp/left_to_classify_ids_Task1.vec']

In [19]:
left_to_classify_ids[0]

'S1137-66272016000300011-1'

In [0]:
# Tokenizzazione e rimozione delle stopword
def tokenize_text(text):
    tokens = list(str(text).lower().split(" "))
    for z in range(0, len(stop_word)):
        if stop_word[z] in tokens:
            while stop_word[z] in tokens:
                tokens.remove(str(stop_word[z]))
    return tokens

In [0]:
# prepare tokenizer
#t = Tokenizer()
#t.fit_on_texts(df['Desc'])
#vocab_size = len(t.word_index) + 1

t = joblib.load("/content/drive/My Drive/codiesp/lstm_fasttext__tokenizer.vec")

In [22]:
#padding text
encoded_left_to_classify = t.texts_to_sequences(left_to_classify)

max_length = 64
padded_left_to_classify = pad_sequences(encoded_left_to_classify, maxlen=max_length, padding='post')
print(padded_left_to_classify[76])

[  687   785   178    49   112   112    42   138  2532  4377  3259  1544
  1226  3056   193   113   785  1465   430   494   756  1218  1937  1465
   312  1317   453 10102   312  1616  8129   149  8589   312     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]


In [0]:
#le = ce.OneHotEncoder(return_df=False, impute_missing=False, handle_unknown="ignore")
#labels = le.fit(list(df['Code']))
le = joblib.load("/content/drive/My Drive/codiesp/lstm_fasttext__label_encoder_le.vec")

In [0]:
# get the vectors
#file = open('/content/drive/My Drive/codiesp/embeddings-l-model_es.vec')

# create a weight matrix for words in training docs
#count = 0
embedding_matrix = joblib.load('/content/drive/My Drive/codiesp/lstm_fasttext__embedding_matrix_medical.vec')
#np.zeros((vocab_size, 300))
#vocab_and_vectors = {}
#arrValues = []
#z = 0
#for line in file:
#    if(z != 0):
#        values = line.split()
#        word = values[0]
#        vector = np.asarray(values[1:], dtype='float32')
#        vocab_and_vectors[word] = vector
#        arrValues.append(vector)
#    else:
#        z= z+1

##for word, i in t.word_index.items():
#    try:
#        embedding_vector = vocab_and_vectors.get(word)#
#
#        if embedding_vector is None:
#            count = count + 1
#            index = random.randint(0, 1000)
#            embedding_vector = arrValues[index]
#    except:
#        count = count+1
#        index = random.randint(0, 1000)
#        embedding_vector = arrValues[index]

#    if embedding_vector is not None:
#        embedding_matrix[i] = embedding_vector
#print(count)

In [25]:
# define the model
vocab_size = len(t.word_index) + 1
input = Input(shape=(64,))
m = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=64, trainable=False) (input)
bi = Bidirectional(LSTM(64, activation ='tanh', return_sequences = True, dropout=0.3)) (m)

aa = SeqSelfAttention(attention_activation='tanh') (bi)
aa = Conv1D(128,5, activation ='relu' ) (aa)
aa = MaxPool1D(2) (aa)
aa = Dropout(0.2) (aa)

added = keras.layers.Concatenate(axis=1)([aa,bi])

ff = GlobalMaxPool1D() (added)
ff = Dense(4000)(ff)
ff = Dropout(0.3) (ff)
ff =Dense(1788, activation='softmax') (ff)

model = keras.models.Model(inputs=[input], outputs=[ff])

model.summary(line_length=100)
model.compile (loss='categorical_crossentropy' , optimizer='adam' , metrics=[ 'accuracy'] )
model.load_weights('/content/drive/My Drive/codiesp/LSTM_CNN_ATT_Fasttext_final_03052020.h5')



Model: "model_3"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 64)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 64, 300)       3275400     input_1[0][0]                    
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 64, 128)       186880      embedding_1[0][0]                
____________________________________________________________________________________________________
seq_self_attention_1 (SeqSelfAtt (None, 64, 128)       8257        bidirectional_1[0][0]            
_________________________________________________________________________

In [26]:
#predict
res = model.predict(padded_left_to_classify)
joblib.dump(res, '/content/drive/My Drive/codiesp/res_Task1.vec')
print(res[0])
#map = le.category_mapping[0]['mapping']
#print(map)
#print(res_encoded)

#labels_map = []
#i = 0
#for a,b in map:
#    labels_map.append(a)

#res_encoded = []
#for a in res:

#    val = a.argmax()
#    res_encoded.append(labels_map[val])

#print(left_to_classify_labels)

[4.5677670e-08 1.9095653e-05 1.6852778e-06 ... 4.1965386e-08 4.6585136e-09
 3.3102005e-08]


In [27]:
from collections import OrderedDict 
max_vals={}
extracted_res = []
extractede_ids = []
map = le.category_mapping[0]['mapping']
labels_map = []
i = 0
for a,b in map:
    labels_map.append(a)

j =0
previous_id = left_to_classify_ids[0]
for result in res:
  id = left_to_classify_ids[j]

  if id != previous_id:
    
    od = OrderedDict(max_vals) 
    #print(str(id)+' '+str(od))

    for key, value in od.items():
      extracted_res.append(value.lower())
      extractede_ids.append(previous_id)
    max_vals = {}
    previous_id = id

  i = 0
  for a in result:
    if a > 0.10:
      max_vals[str(a)] = labels_map[i]
    i = i+1
  if len(max_vals) == 0:
    v = result.max()
    index = result.argmax()
    max_vals[str(v)] = labels_map[index]

  j = j+1

#Last item
od = OrderedDict(max_vals) 

for key, value in od.items():
    extracted_res.append(value.lower())
    extractede_ids.append(previous_id)

print(extracted_res)
print(extractede_ids)
  

['r68.84', 'r52', 'r52', 'm89.9', 'c79.51', 'q04.0', 'c79.51', 'm89.9', 'd16.5', 'l90.5', 'r60.9', 'k13.79', 'c41.1', 't14.90', 't30.0', 'm89.9', 'm89.70', 'c77.9', 'r58', 'z93.0', 'r52', 'c79.51', 'z87.891', 'd64.9', 'e83.51', 'r50.9', 'd30.02', 'r23.1', 'h05.20', 'h02.402', 'g82.20', 'k13.79', 's02.5xx', 'h05.20', 'h53.2', 'h55.00', 'n28.89', 'h21.9', 'd12.1', 'j34.1', 'r63.4', 'r11.10', 'r11.0', 'z87.891', 'i10', 'k76.81', 'i10', 'k76.81', 'z79.01', 'n28.9', 'r69', 'n26.1', 'm85.50', 'n28.9', 'r69', 'k76.9', 'r63.4', 'r53.1', 'd64.9', 'r41.0', 'b96.20', 'm62.50', 'n28.9', 'n17.9', 'r31.9', 'r31.0', 'n40.0', 'k13.79', 'n32.9', 'c67.9', 'd64.9', 'n40.0', 'r31.9', 'r31.0', 'n32.9', 'k11.9', 'l98.9', 'k11.9', 'r23.4', 'm32.9', 'd72.822', 'd49.7', 'i96', 'r74.0', 'b15.9', 'r59.9', 'd72.829', 'o80', 'h53.40', 'd72.829', 'r11.10', 'k92.0', 'b97.6', 'n50.9', 'n48.89', 'a63.0', 'k11.9', 'd64.9', 'd64.9', 'r11.10', 'd50.0', 'l98.9', 'd64.9', 'i10', 'e11.9', 'r52', 'm54.2', 'r60.9', 'r52', 'd6

In [0]:
file = open('task1_run1_bert_01_multiclass_deep_net.tsv','w+')

for i in range(0, len(extracted_res)):
  file.write(str(extractede_ids[i])+'\t'+str(extracted_res[i])+'\n')
  file.flush()

file.close()

In [29]:
print('Testing accuracy %s' % accuracy_score(left_to_classify_labels, res_encoded))
print('Testing F1 score: {}'.format(f1_score(left_to_classify_labels, res_encoded, average='weighted')))

from sklearn.metrics import classification_report
#print(classification_report(left_to_classify_labels, res_encoded,digits=5))

NameError: ignored

#Merge back the results


In [0]:
final_res = []

i = 0
for item in res_encoded_original:
  if item == 0:
    final_res.append('emp')
  else:
    final_res.append(res_encoded[i])
    i = i+1


In [0]:
print('Testing accuracy %s' % accuracy_score(test_bal, final_res))
print('Testing F1 score: {}'.format(f1_score(test_bal, final_res, average='weighted')))
print(classification_report(test_bal, final_res,digits=5))